In [2]:
import nltk
s1 = 'דיכאון'
s2 = 'דכאון'
nltk.edit_distance(s1, s2, transpositions=False)

1

In [3]:
import re

In [4]:
typo_word= 'בטיחון'

In [5]:
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('bgulex.utf8.hr', 'r', encoding='utf8') as f:
    text = f.read()

In [6]:
lines = text.split('\n')

In [7]:
words_in_heb_dict = set()
for line in lines:
    word = line.split(':')[0].split(' ')[0].strip().replace(' DEF','')
    if len(word) > 0:
        words_in_heb_dict.add(word)

In [8]:
nearest_words_in_keyboard = {'א' : 'רכעיט',
'ב' : 'סדגכה',
'ג' : 'רכהבסד',
'ד' : 'גבסזש',
'ה' : 'בגכענ',
'ו' : 'םךלחו',
'ז' : 'סדש',
'ח' : 'יטוןלתצמ',
'ט' : 'אעחו',
'י' : 'אטוחצמנע',
'כ' : 'קראענהבג',
'ל' : 'חוןםךצ',
'מ' : 'ניחצ',
'נ' : 'מיעה',
'ס' : 'זשדגב',
'ע' : 'כראטינה',
'פ' : 'םלך',
'צ' : 'מחלת',
'ק' : 'רכגד',
'ר' : 'קגכעא',
'ש' : 'דסז',
'ת' : 'צלךץ',
'ם' : 'ןלךףפ',
'ן' : 'וחלךם',
'ץ' : 'ףךת',
'ף' : 'פםךץ',
'ך' : 'ףפםןלת'}

In [9]:
def generate_words_that_has_nearest_letters_in_keyboard(nearest_words_in_keyboard,word,number_letters=1):
    closest_words = list()
    for letter_index in range(len(word)):
        letter = word[letter_index]
        for near_letters in nearest_words_in_keyboard[letter]:
            new_word = str(word[:letter_index] + near_letters +word[letter_index+1:])
            closest_words.append(new_word)
            if number_letters > 1:
                for new_word_ in fix_by_nearest_letters_in_keyboard(nearest_words_in_keyboard,new_word,number_letters-1):
                    closest_words.append(new_word_)
    for close_word in set(closest_words):
        yield close_word

In [10]:
def permutations(string,number_swaps=1,location=0):
    # everything to the right of step has not been swapped yet
    prem_words = []
    for i in range(location,len(string)-1):

        # copy the string (store as array)
        string_copy = [character for character in string]

        # swap the current index with the step
        string_copy[i], string_copy[i+1] = string_copy[i+1], string_copy[i]

        prem_words += [''.join(string_copy)]
        if number_swaps > 1:
            for i in permutations(string_copy,number_swaps-1,location+3):
                prem_words += [i]
    for i in set(prem_words):
        yield i


In [11]:
def generate_transpory_words(word,heb_dict):
    perms = [''.join(p) for p in permutations(word)]
    for perm in perms:
        if perm in heb_dict:
            yield perm

In [12]:
for i in generate_transpory_words(typo_word,words_in_heb_dict):
    print(i)

ביטחון


In [13]:
def find_closest_words(heb_dict,distance_word):
    distance = -1
    closest_words = list()
    for word in heb_dict:
        curr_distance = nltk.edit_distance(word, distance_word, transpositions=True)
        if distance == curr_distance:
            closest_words += [word]
        if distance > curr_distance or distance == -1:
            distance = curr_distance
            closest_words = [word]
    closest_words = sorted(closest_words, key=len)
    return closest_words, distance

In [14]:
def common_typo_hebrew(word, start_index=0):
    common_words = list()
    for letter_index in range(start_index, len(word)):
        letter = word[letter_index]
        near_letters = [letter]
        letter = word[letter_index]
        if letter == 'כ':
            near_letters = ['ח','ק']
        if letter == 'ח':
            near_letters = ['ק','כ']
        if letter == 'ק':
            near_letters = ['ח','כ']
        if letter == 'ש':
            near_letters = ['ס']
        if letter == 'ס':
            near_letters = ['ש']
        if letter == 'ו':
            near_letters = ['ב']
        if letter == 'ב':
            near_letters = ['ו']
        if letter == 'א':
            near_letters = ['ע','ה']
        if letter == 'ך':
            near_letters = ['ח']
        if letter == 'ה':
            near_letters = ['ע','א']
        if letter == 'ע':
            near_letters = ['א','ה']
        if letter == 'ט':
            near_letters = ['ת']
        if letter == 'ת':
            near_letters = ['צ']
        if near_letters[0] != letter:
            for near_letter in near_letters:
                new_word = str(word[:letter_index] + near_letter +word[letter_index+1:])
                common_words += [new_word]
                common_words += common_typo_hebrew(new_word,letter_index+1)
    for common_word in common_words:
        yield common_word

In [15]:
%time find_closest_words(words_in_heb_dict, typo_word)

CPU times: user 28.5 s, sys: 281 ms, total: 28.8 s
Wall time: 31.6 s


(['בטיחו', 'בטיחן', 'בטחון', 'בטיחות', 'ביטחון', 'בטיחכן', 'בטיחותן'], 1)

In [16]:
def remove_emojies(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return str(emoji_pattern.sub(r'', text))

In [17]:
problem_words = ['מתשמש','היתקבל','אזרחןת','מדריח','וקמנדו','היתגייס','מוצאש','אפסדים','גמילים','פסכיאטר','אישתתף','קיריה','תאבחון','תחיילתם','לירפא','משיהו','הל''צ','אווירנאוטיקה','מעונין','פסיכיטאר','מישוא','ויד','דיגטלי','ריאיונות','צה\'ל','פסיכאטריות','יפואלרגני']

In [18]:
for word in set(problem_words):
    for i in generate_transpory_words(word,words_in_heb_dict):
        print(f'{word} ------> {i}')

ויד ------> יוד
פסיכיטאר ------> פסיכיאטר
מעונין ------> מעוינן
מתשמש ------> תמשמש
מתשמש ------> משתמש
משיהו ------> מישהו
אזרחןת ------> אזרחתן
הלצ ------> הצל
קיריה ------> קרייה
קיריה ------> יקריה
וקמנדו ------> קומנדו
גמילים ------> גימלים


In [19]:
for word in set(problem_words):
    if len(word) > 1:
        typo_word = remove_emojies(word)
        #print(typo_word)
        try:
            for i in generate_words_that_has_nearest_letters_in_keyboard(nearest_words_in_keyboard,typo_word):
                if i in words_in_heb_dict:
                    print(f'{typo_word} ------> {i}')
        except:
            #print(typo_word)
            pass

ויד ------> ליד
ויד ------> ועד
מעונין ------> מעונים
מעונין ------> מעוננן
מעונין ------> מעויין
משיהו ------> משיבו
משיהו ------> משיכו
משיהו ------> משינו
משיהו ------> משחהו
משיהו ------> משנהו
משיהו ------> משיגו
מישוא ------> מישור
אזרחןת ------> אזרחות
הלצ ------> הלל
הלצ ------> הלח
היתקבל ------> המתקבל
קיריה ------> קוריה
קיריה ------> כיריה
קיריה ------> דיריה
קיריה ------> גיריה
קיריה ------> קיאיה
קיריה ------> ריריה
קיריה ------> קמריה
קיריה ------> קצריה
קיריה ------> קטריה
גמילים ------> גמולים
גמילים ------> המילים
היתגייס ------> המתגייס
מדריח ------> מבריח
מדריח ------> מסריח


In [20]:
for word in set(problem_words):
    for i in common_typo_hebrew(word):
        if i in words_in_heb_dict:
            print(f'{word} ------> {i}')

ויד ------> ביד
משיהו ------> מסיעו
משיהו ------> משיאו
מישוא ------> מישבה
אפסדים ------> הפסדים
היתקבל ------> איתחול
קיריה ------> כיריה


In [28]:
file_type = 'thread'
field_name = 'title'

In [29]:
import pandas as pd
df = pd.read_csv(f'{file_type}_with_yap.csv')

In [44]:
for problematic_word in problem_words:
    print(f'the problematic word is {problematic_word}')
    print(list(df.loc[df[f'{field_name}'].str.contains(problematic_word)][f'{field_name}']))

the problematic word is מתשמש
['לא יכול להיכנס למתשמש של מתגייסים - FXP']
the problematic word is היתקבל
['עם ההיתי בכלא צבאי על סמים אני יוכל להיתקבל באזרחות לעבודה עם - FXP']
the problematic word is אזרחןת
['כדורים מפסיכאטר יכתב לי באזרחןת - FXP']
the problematic word is מדריח
['דרישות במיון למדריח כושר גופני - FXP']
the problematic word is וקמנדו
['סיירת שיריון מול חטיבת הוקמנדו - FXP']
the problematic word is היתגייס
['כל האנשים ששלחו טופס 55 לקרבי וצריכים להיתגייס בנובמבר-דצמבר - FXP', 'כל האנשים ששלחו טופס 55 לקרבי וצריכים להיתגייס בנובמבר-דצמבר - FXP', 'לכל אלה שעומדים להיתגייס בחודש הקרוב או בחודשיים הקרובים - FXP', 'מה אתם מעדיפים להיתגייס או לעשות שנת שירות - FXP', 'מחר יש לי שיחה עם קצין מיון גברים לגיוס... ולא בא לי להיתגייס - FXP', 'מסוכן להיתגייס לשבס? - FXP', 'רוצה להיתגייס לתותחנים - FXP']
the problematic word is מוצאש
['ביקור רופא בראשון לציון פתוח עד 1 במוצאש בטוח נכון ? - FXP']
the problematic word is אפסדים
['צהל צריך כאלה לאפסדים - FXP']
the problematic word is גמי